# 2020_10_15:Dimensionality Reduction Methods



FAILED using TruncatedSVD/LatentSentimentAnalysis. Maximum dimension achieved before 32GB RAM ceiling = 400. Maximum training F1 score ~ 47%.

Going for semiautomatic chi-square based analysis.

# Imports

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt  
import wordcloud as wc 
import seaborn as sns 
import nltk
import re
from tqdm import tqdm
tqdm.pandas()

train_dset_df = pd.read_csv("./dataset/pp/2020_10_04_preprocessed_train.csv")
test_dset_df = pd.read_csv("./dataset/pp/2020_10_04_preprocessed_test.csv")
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(eval)
test_dset_df["preprocessed"] = test_dset_df["preprocessed"].progress_apply(eval)
train_dset_df = train_dset_df.drop(axis="columns", labels="Unnamed: 0")
test_dset_df = test_dset_df.drop(axis="columns", labels="Unnamed: 0")
from tqdm import tqdm 
tqdm.pandas()
import gc

100%|██████████| 522449/522449 [00:07<00:00, 70949.80it/s]


# Snowball Stemming

In [3]:
from nltk.stem import SnowballStemmer

In [4]:
snow_stem = SnowballStemmer("english")

In [5]:
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(lambda x: [snow_stem.stem(i) for i in x])

100%|██████████| 783673/783673 [00:59<00:00, 13113.50it/s]


In [6]:
test_dset_df["preprocessed"] = test_dset_df["preprocessed"].progress_apply(lambda x: [snow_stem.stem(i) for i in x])

100%|██████████| 522449/522449 [00:39<00:00, 13171.13it/s]


In [7]:
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(lambda x: " ".join(x))

100%|██████████| 783673/783673 [00:00<00:00, 1025795.22it/s]


In [8]:
test_dset_df["preprocessed"] = test_dset_df["preprocessed"].progress_apply(lambda x: " ".join(x))

100%|██████████| 522449/522449 [00:00<00:00, 890153.65it/s]


In [9]:
train_dset_questions = train_dset_df["preprocessed"]
test_dset_questions = test_dset_df["preprocessed"]

questions = pd.concat([train_dset_questions, test_dset_questions])

# TFIDFVectorizer

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [11]:
vectorizer = CountVectorizer()

In [12]:
vectorizer.fit(train_dset_questions)

CountVectorizer()

In [13]:
sparse_x_train = vectorizer.transform(train_dset_df["preprocessed"])

In [14]:
sparse_x_train

<783673x69405 sparse matrix of type '<class 'numpy.int64'>'
	with 5499037 stored elements in Compressed Sparse Row format>

# Dimensionality reduction MK II

In [15]:
# sparse_x_train.shape

In [15]:
# sparse_x_train_columns = sparse_x_train.tocsc()

In [16]:
# y = train_dset_df["target"].to_numpy()
# p_y = np.sum(y)/y.shape[0]

In [17]:
# filter_delta = 0.2
# columns_to_keep = []

In [18]:
# for i in tqdm(range(sparse_x_train.shape[1])):
#     curr_column  = sparse_x_train_columns[:,i].toarray().reshape(-1, )
#     curr_column = (curr_column > 0).astype(int)
#     n_curr_col_one = curr_column.sum()
#     n_curr_col_zero = curr_column.shape[0] - n_curr_col_one
#     n_y_and_curr_col = np.sum(np.logical_and(curr_column, y).astype(int))
#     n_y_and_not_curr_col = np.sum(np.logical_and(1-curr_column, y).astype(int))
#     p_y_if_curr_col = n_y_and_curr_col/n_curr_col_one
#     p_y_if_not_curr_col = n_y_and_not_curr_col/n_curr_col_zero
#     if(abs(p_y_if_curr_col - p_y) > filter_delta or abs(p_y_if_not_curr_col - p_y) >filter_delta):
#         columns_to_keep.append(i)

In [19]:
# len(columns_to_keep)

In [20]:
# reduced_sparse_x_train = sparse_x_train[:,columns_to_keep]

In [21]:
# from sklearn.decomposition import TruncatedSVD

In [22]:
# tsvd = TruncatedSVD(n_components = 400, random_state=42)

In [23]:
# tsvd.fit(sparse_x_all)

# Training

In [24]:
# sparse_x_train = vectorizer.transform(train_dset_df["preprocessed"])
# transformed_x_train = tsvd.transform(sparse_x_train)

In [25]:
# transformed_x_train.shape

In [1]:
# from sklearn.kernel_approximation import Nystroem
from sklearn.svm import LinearSVC, SVC

In [72]:
svm = SVC(verbose=2, max_iter = -1, class_weight= {0:1, 1:1000},kernel="linear")

In [73]:
from sklearn.ensemble import BaggingClassifier

In [84]:
bagger = BaggingClassifier(base_estimator=svm, n_estimators=50, max_samples=15000, n_jobs=6, random_state=42,verbose=2)

In [85]:
bagger.fit(sparse_x_train, train_dset_df["target"])

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:  1.6min remaining:  1.6min
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:  1.7min finished


BaggingClassifier(base_estimator=SVC(class_weight={0: 1, 1: 1000},
                                     kernel='linear', verbose=2),
                  max_samples=15000, n_estimators=50, n_jobs=6, random_state=42,
                  verbose=2)

In [86]:
yhat = bagger.predict(sparse_x_train)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed: 28.0min remaining: 28.0min
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed: 30.0min finished


In [77]:
train_dset_df["yhat"] = yhat

In [78]:
from sklearn.metrics import f1_score

In [79]:
wrongs = train_dset_df[train_dset_df["yhat"] != train_dset_df["target"]]

In [80]:
y = train_dset_df["target"]

In [81]:
f1_score(y, yhat)

0.5203946649837983

In [82]:
[(name, len(x)) for name, x in wrongs.groupby(by="target")]

[(0, 12508), (1, 27011)]

In [83]:
[(name, len(x)) for name, x in train_dset_df.groupby(by="target")]

[(0, 735222), (1, 48451)]

# Testing and writing output

In [57]:
sparse_x = vectorizer.transform(test_dset_df["preprocessed"])

In [58]:
reduced_sparse_x = sparse_x[:,columns_to_keep]

In [59]:
yhat = svm.predict(reduced_sparse_x)

In [60]:
test_dset_df.question_text = yhat

In [61]:
test_dset_df = test_dset_df.drop(axis="columns", labels="preprocessed")
test_dset_df.head()

,qid,question_text
0,f56a9a31974dc66186e8,0
1,d957c3758060f45da303,0
2,ad822d5abaedb9e247b9,0
3,4e979c23eeb6a4bd1f2e,0
4,333cc031262566b8da49,0


In [62]:
test_dset_df = test_dset_df.rename(columns={"qid":"qid", "question_text":"target"})
test_dset_df.target = test_dset_df.target.apply(round)

In [63]:
test_dset_df.to_csv("./outputs/2020_10_16_a_testset_output.csv", index=False)